# O modelo SIR com PySINDy

$$\frac{dS}{dt} = -\beta S I$$

$$\frac{dI}{dt} = \beta S I - \gamma I,$$

$$\frac{dR}{dt} =  \gamma I$$


$\beta=$0.3 e $\gamma=$0.1

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import pysindy as ps
from pysindy.differentiation import FiniteDifference


(CVXPY) Jan 10 02:16:04 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: Não foi possível encontrar o módulo especificado.')


In [2]:
np.random.seed(100)
def sir(t, x, beta=0.3, gamma=0.1):
    return [
        -beta * x[0] * x[1],
        beta * x[0] * x[1] - gamma*x[1],
        gamma*x[1],
    ]


t_start = 0.0
t_end = 100.0
t_step =  0.002
t_span = (t_start, t_end)

t_eval = np.arange(t_start, t_end, t_step)   # 
# Criando o vetor de condições iniciais
N = 1
I0, R0 = 0.001, 0.0
S0 = N - I0 - R0
x0_train = np.array([S0, I0, R0]) 

solution = solve_ivp(sir, t_span, x0_train, t_eval=t_eval)

t = solution.t
pop = solution.y.T


In [3]:
# Definição das funções personalizadas
library_functions = [lambda x: x, lambda x, y: x * y, lambda x, y: x + y]
library_function_names = [lambda x: x, lambda x, y: f"{x} * {y}", lambda x, y: f"{x} + {y}"]

feature_library= ps.CustomLibrary(library_functions=library_functions, 
                                  function_names=library_function_names).fit(solution.y.T)
feature_library.transform(solution.y.T)
feature_library.get_feature_names()


differentiation_method = ps.FiniteDifference(order=2)
optimizer = ps.STLSQ(threshold=0.6, alpha=1e-4, normalize_columns=True)
model = ps.SINDy(differentiation_method=differentiation_method,
                 feature_library=feature_library,
                 feature_names = ['S', 'I', 'R'],
                 optimizer=optimizer,)

model.fit(solution.y.T, solution.t, quiet=True)
model.print()

(S)' = -0.301 S * I
(I)' = -0.100 I + 0.300 S * I
(R)' = 0.100 I
